In [ ]:
from datetime import datetime, timedelta

# Notebook parameters

In [ ]:
sample_day = (datetime.now().date() - timedelta(days=1)).strftime('%Y-%m-%d')
source_folder1 = 'gs://dsart_nearline1/pipelines/gambit1/'
source_folder2 = 'gs://dsart_nearline1/pipelines/snapshots/engagement8h/'
source_folder3 = 'gs://dsart_nearline1/pipelines/snapshots/links/'
target_folder = 'gs://dsart_nearline1/pipelines/dataset1/'

In [ ]:
sample_day

# Dependencies

In [ ]:
import os
import pandas
import time
import numpy
import json
from dotenv import load_dotenv

# Load data

In [ ]:
def clean_column_names(df):
    r = {}
    for x in list(df.columns):
        if '\r' in x:
            r[x] = x.replace('\r', '')
    if len(r)>0:        
        print('renaming', r)
        df.rename(columns=r, inplace=True)

In [ ]:
file1 = sample_day+'.csv'
file1_source = source_folder1 + file1
file1_local = sample_day+'_gambit1.csv'
file1_source, file1_local

In [ ]:
! gsutil cp {file1_source} {file1_local}

In [ ]:
df1 = pandas.read_csv(file1_local, lineterminator='\n')

In [ ]:
clean_column_names(df1)

In [ ]:
df1

In [ ]:
file2 = sample_day+'.csv'
file2_source = source_folder2 + file2
file2_local = sample_day+'_eng.csv'
file2_source, file2_local

In [ ]:
! gsutil cp {file2_source} {file2_local}

In [ ]:
df2 = pandas.read_csv(file2_local, lineterminator='\n')

In [ ]:
clean_column_names(df2)

In [ ]:
df2

In [ ]:
file3 = sample_day+'.csv'
file3_source = source_folder3 + file3
file3_local = sample_day+'_links.csv'
file3_source, file3_local

In [ ]:
! gsutil cp {file3_source} {file3_local}

In [ ]:
df3 = pandas.read_csv(file3_local, lineterminator='\n')

In [ ]:
clean_column_names(df3)

In [ ]:
df3

# Merge dataset and perform some checks

In [ ]:
df = df1.merge(df2, on='id', how='left').merge(df3, on='fid', how='left')
df

In [ ]:
# Check number of casts missing in engagement snapshot
df['num_like'].isnull().sum()

In [ ]:
# Check number of fids missing in links snapshot
df['link_from_add'].isnull().sum()

In [ ]:
# Check that cast hashes always match (except when missing)
(df['hash']!=df['cast_hash']).sum()

In [ ]:
del df['cast_hash']

In [ ]:
df['day'] = sample_day

In [ ]:
df['text_len'] = df['text'].str.len()

In [ ]:
df['num_follower'] = (df['link_to_add'] - df['link_to_del']).fillna(0)
df['num_follower'].describe()

In [ ]:
# Some have been unfollowed more than followed, set to zero
print((df['num_follower']<0).sum())
df.loc[df['num_follower']<0, 'num_follower'] = 0
df['num_follower'].describe()

In [ ]:
df['num_following'] = (df['link_from_add'] - df['link_from_del']).fillna(0)
df['num_following'].describe()

In [ ]:
# Some have unfollowed more than followed, set to zero
print((df['num_following']<0).sum())
df.loc[df['num_following']<0, 'num_following'] = 0
df['num_following'].describe()

In [ ]:
# Fill in NAs in engagement table

In [ ]:
df['num_like'] = df['num_like'].fillna(0)
df['num_like'].describe()

In [ ]:
df['num_recast'] = df['num_recast'].fillna(0)
df['num_recast'].describe()

In [ ]:
df['num_reply'] = df['num_reply'].fillna(0)
df['num_reply'].describe()	

# Define targets

In [ ]:
account_too_big = df['num_follower'].quantile(0.99)
account_too_big

In [ ]:
df = df[df['num_follower']<account_too_big].copy()
len(df)

In [ ]:
num_bins = 20

In [ ]:
num_follower_bin, _ = pandas.qcut(df['num_follower'], num_bins, retbins=True, labels=False, duplicates='drop')
df['num_follower_bin'] = num_follower_bin

In [ ]:
agg1 = df.groupby('num_follower_bin').agg({
    'num_follower':['count', 'min', 'mean', 'max'], 
    'text_len':['mean'], 
    'num_like':['mean', 'max'],
    'num_recast':['mean', 'max'],
    'num_reply':['mean', 'max']})
agg1

In [ ]:
pandas.DataFrame({
    'num_follower': agg1['num_follower']['mean'],
    'num_like': agg1['num_like']['mean'],
    'num_recast': agg1['num_recast']['mean'],
    'num_reply': agg1['num_reply']['mean']
}).plot(x='num_follower', y=['num_like', 'num_recast', 'num_reply'])

In [ ]:
text_len_bin, _ = pandas.qcut(df['text_len'], num_bins, retbins=True, labels=False, duplicates='drop')
df['text_len_bin'] = text_len_bin

In [ ]:
agg2 = df.groupby('text_len_bin').agg({
    'num_follower':['count', 'min', 'mean', 'max'], 
    'text_len':['mean'], 
    'num_like':['mean', 'max'],
    'num_recast':['mean', 'max'],
    'num_reply':['mean', 'max']})
agg2

In [ ]:
pandas.DataFrame({
    'text_len': agg2['text_len']['mean'],
    'num_like': agg1['num_like']['mean'],
    'num_recast': agg1['num_recast']['mean'],
    'num_reply': agg1['num_reply']['mean']
}).plot(x='text_len', y=['num_like', 'num_recast', 'num_reply'])

In [ ]:
df_target = []
cols_target = ['id', 'num_follower', 'num_like', 'num_recast', 'num_reply', 'hash']

In [ ]:
def target_top(df_tmp, label, fraction):
    df_tmp[label] = 0
    top = int(len(df_tmp)*fraction)
    df_tmp.loc[df_tmp.index[:top], label] = 1

In [ ]:
def make_bin_target(df_tmp):
    df_tmp.sort_values('hash', inplace=True)
    df_tmp.sort_values('num_follower', inplace=True, ascending=True)
    df_tmp.sort_values(['num_like', 'num_recast', 'num_reply'], inplace=True, ascending=False)
    df_tmp.reset_index(inplace=True, drop=True)
    target_top(df_tmp, 'target_05', 0.05)
    target_top(df_tmp, 'target_10', 0.10)
    target_top(df_tmp, 'target_15', 0.15)
    target_top(df_tmp, 'target_20', 0.20)
    target_top(df_tmp, 'target_25', 0.25)
    del df_tmp['num_follower']
    del df_tmp['num_like']
    del df_tmp['num_recast']
    del df_tmp['num_reply']
    del df_tmp['hash']
    return df_tmp

In [ ]:
num_follower_bins = len(df['num_follower_bin'].unique())
num_follower_bins

In [ ]:
text_len_bins = len(df['text_len_bin'].unique())
text_len_bins

In [ ]:
for i in range(0, num_follower_bins):
    df_i = df[df['num_follower_bin']==i].copy()
    for j in range(0, text_len_bins):
        df_j = df_i[df_i['text_len_bin']==j][cols_target].copy()
        df_tmp = make_bin_target(df_j)
        df_target.append(df_tmp)
len(df_target)

In [ ]:
df_target = pandas.concat(df_target, axis=0)
df_target

In [ ]:
df_target[['target_05', 'target_10', 'target_15', 'target_20', 'target_25']].mean()

In [ ]:
df = df.merge(df_target, on='id')
df

In [ ]:
df.groupby('target_05').agg({
    'text_len':['mean'],
    'text_len_bin':['mean'],
    'num_follower':['mean'], 
    'num_follower_bin':['mean'], 
    'num_like':['mean']
})

In [ ]:
df.groupby('target_10').agg({
    'text_len_bin':['mean'],
    'num_follower_bin':['mean'], 
    'num_like':['mean']
})

In [ ]:
df.groupby('target_15').agg({
    'text_len_bin':['mean'],
    'num_follower_bin':['mean'], 
    'num_like':['mean']
})

In [ ]:
df.groupby('target_20').agg({
    'text_len_bin':['mean'],
    'num_follower_bin':['mean'], 
    'num_like':['mean']
})

In [ ]:
df.groupby('target_25').agg({
    'text_len_bin':['mean'],
    'num_follower_bin':['mean'], 
    'num_like':['mean']
})

# Final Dataframe

In [ ]:
questions = [
'q_clear',
'q_audience',
'q_info',
'q_easy',
'q_verifiable',
'q_personal',
'q_funny',
'q_meme_ref',
'q_emo_res',
'q_happiness',
'q_curiosity',
'q_aggressivity',
'q_surprise',
'q_interesting_ask',
'q_call_action'
]
len(questions)

In [ ]:
categories = [
'c_arts',
'c_business',
'c_crypto',
'c_culture',
'c_misc',
'c_money',
'c_na',
'c_nature',
'c_politics',
'c_sports',
'c_tech_science'
]
len(categories)

In [ ]:
topics = [
't_advertising','t_analysis','t_animals','t_artificial_intelligence',
't_basketball','t_bitcoin','t_blockchain','t_books',
't_cats','t_chess','t_coding','t_comedy',
't_data','t_defi','t_degen','t_design','t_dogs',
't_economy','t_elections','t_ethereum',
't_family','t_farcaster','t_fashion','t_finance','t_fitness','t_food','t_football','t_founders',
't_gaming','t_greetings','t_growth',
't_health','t_history','t_holidays',
't_jobs',
't_lifestyle',
't_marketing','t_markets','t_maths','t_movies','t_music',
't_na','t_news','t_nft',
't_other','t_outdoors',
't_people','t_photography','t_physics','t_prices',
't_real_estate','t_roasting',
't_science','t_security','t_shopping','t_solana',
't_technology','t_tennis','t_trading','t_travel',
't_weather'
]
len(topics)

In [ ]:
df['category'] = numpy.argmax(df[categories], axis=1)

In [ ]:
df['category_label'] = df['category'].apply(lambda x: categories[x])

In [ ]:
df['topic'] = numpy.argmax(df[topics], axis=1)

In [ ]:
df['topic_label'] = df['topic'].apply(lambda x: topics[x])

In [ ]:
if 'timestamp' not in df.columns:
    df['timestamp'] = df['day'] 

In [ ]:
first = [
    'day',
    'timestamp',
    'id',
    'hash',
    'fid',
    'text',
    'text_len',
    'category',
    'category_label',
    'topic',
    'topic_label',
    'num_like',
    'num_recast',
    'num_reply',
    'link_from_add',
    'link_from_del',
    'link_to_add',
    'link_to_del',
    'num_follower',
    'num_following',
    'num_follower_bin',
    'text_len_bin',
    'target_05',
    'target_10',
    'target_15',
    'target_20',
    'target_25',
    'dim_1',
    'dim_2',
    'dim_3' 
]
len(first)

In [ ]:
all_columns = first + questions + categories + topics
len(all_columns)

In [ ]:
dropped_columns = [x for x in df.columns if x not in all_columns]
dropped_columns

In [ ]:
df = df[all_columns]
df

# Look at topics classification

In [ ]:
for i in range(len(categories)):
    c = categories[i]
    df_t = df[df['category']==i].sort_values(['num_like', 'num_recast', 'num_reply', c], ascending=False)[:10]
    print('*'*32, i, c, '*'*32)
    for _, row in df_t.iterrows():
        print('-'*16, row['topic_label'], '-'*16)
        print(row['text'])
        print('-'*16, row['num_like'], row['num_recast'], row['num_reply'], '-'*16)
    print()    
    print()

# Look at questions features

In [ ]:
for i in range(len(questions)):
    q = questions[i]
    df_q = df.sort_values(q)
    print('*'*32, i, q, '*'*32)
    for _, row in df_q[:5].iterrows():
        print('-'*16, 'low', row[q], '-'*16)
        print(row['text'])
    for _, row in df_q[-5:].iterrows():
        print('-'*16, 'high', row[q], '-'*16)
        print(row['text'])
    print()    
    print()

# Save output

In [ ]:
output_file = sample_day + '.csv'
output_file

In [ ]:
df.to_csv(output_file, index=False, float_format='%.2f')

In [ ]:
! gsutil cp {output_file} {target_folder}